# 실제 데이터로 작업
- 머신러닝 학습 시 실제 데이터로 실험
  - 유명 공개 데이터 저장소: UC 얼바인 머신러닝 저장소, 캐글 데이터셋, AWS 데이터셋
  - 메타 포털: 데이터 포털, 오픈 데이터 모니터, 퀀들, Quora, 데이터셋 서브레딧, 위키백과 머신러닝 데이터셋 목록
- 캘리포니아 주택 가격 데이터셋 사용(범주형 특성 추가, 몇개 특성 제외)

# 큰 그림 보기
- 예제: 캘리포니아 인구조사 데이터를 사용해 주택 가격 모델을 만들기.
(블록 그룹마다 인구, 중간 소득, 중간 주택 가격 등이 있는 데이터)
## 1. 문제 정의
- 비즈니스의 목적을 아는 것: 문제를 어떻게 구성할지, 어떤 알고리즘을 선택할지, 모델 평가에 어떤 성능 지표를 사용할지, 모델 튜닝을 위해 얼마나 노력을 투여할지 결정
  - 예제: 여러 가지 다른 신호와 함께 해당 지역 투자 분석 후 투자 여부를 결정
> 파이프라인: 데이터 처리 컴포넌트들의 연속, 보통 비동기적으로 동작하며 독립적
- 현재 솔루션이 어떻게 구성되어 있는지: 문제 해결 방법에 대한 정보, 참고 성능으로 사용
  - 예제: 수동으로 추정(최신 정보를 모음, 얻을 수 없을때는 복잡한 규칙을 사용해 추정)
- 문제 정의: 레이블된 샘플->지도학습, 값의 예측에 사용할 특성이 여러개->다중 회귀, 단변량 회귀, 연속적인 흐름이 없고 데이터를 메모리에 적재할만한 크기->배치 학습
## 2. 성능 측정 지표 선택
- 회귀 문제의 전형적인 성능 지표: 평균 제곱근 오차(RMSE)
  - $RMSE(\bf X,\rm h)=\sqrt{\frac{1}{m} \sum_{i=1}^m(h(\bf x\rm ^{(i)} - y^{(i)})^2}$
    - m: 데이터셋에 있는 샘플 수
    - $\bf x^{(i)}$: i번째 샘플의 전체 특성값의 벡터
    - $y^{(i)}$: i번쨰 샘플의 레이블(해당 샘플의 기대 출력값)
    - $\bf X$: 데이터셋에 있는 모든 샘플의 모든 특성값을 포함하는 행렬(레이블 제외)
    - h: 예측 함수(가설)
    - RMSE(X, h): 가설h를 사용하여 일련의 샘플을 평가하는 비용함수
- 이상치로 보이는 구역이 많은 경우, 평균 절대 오차(평균 절대 편차)를 고려 가능
  - $MAE(\bf X, \rm h)=\frac{1}{m}\sum_{i=1}^m|h(\bf x\rm ^{(i)})-y^{(i)}|$
- 예측값의 벡터와 타깃값의 벡터 사이의 거리를 재는 방법(노름)
  - 유클리디안 노름: L2노름, 제곱항을 합한 것의 제곱근
  - 맨해튼 노름: L1노름, 절댓값의 합을 계산
  - 원소가 n개인 벡터 $\bf v$의 Lk노름: 
  $||\bf v||_k = (|v_0|^k+|v_1|^k+...+|v_n|^k)^{^1/_k}$
  - 노름의 지수가 클수록 큰 값의 원소에 치우치며 작은 값은 무시됨(이상치에 민감)
## 3. 가정 검사
- 시스템이 출력이 하위 시스템의 입력으로 연결될 때 가정을 검사할 필요O

# 데이터 가져오기
## 1. 작업환경 만들기

```python
# 머신러닝 코드와 데이터셋을 저장할 작업 디렉터리 생성
$ export ML_PATH = "$HOME/ml"
$ mkdir -p $ML_PATH

# 필요한 파이썬 패키지 설치 - 주피터, 판다스, 맷플롯립, 사이킷런
$ pip3 --version
$ python3 -m pip install --user -U pip

# 독립된 개발환경 만들기(pip로 virtualenv를 설치)
$ python3 -m pip install --user -U virtualenv
$ cd $ML_PATH
$ virtualenv env
$ .\my_env\Scripts\activate #윈도우에서 활성화
$ source env/bin/acticate #맥,리눅스에서 활성화

# pip명령으로 필요한 패키지들을 설치
$ pip3 install --upgrade jupyter matplotlib numpy pandas scipy scikit-learn
$ python3 -m ipykernel install --user --name=python3 #주피터에 커널 등록
$ jupyter notebook
```

## 2. 데이터 다운로드

- 일반적으로 다룰 데이터는 관계형 데이터 베이스에 들어있음(여러 테이블, 문서, 파일로 나뉘어 있음)
  - 보안 자격, 접근 권한이 필요
  - 데이터의 구조를 잘 알아야 함
- 실습 데이터는 모든 데이터가 하나의 CSV파일(이를 압축한 .tgz)에 들어있음
  - 간단한 함수를 만들어 데이터를 다운로드 -> 여러 기기에 데이터셋을 설치해야할 때 편리함.


In [5]:
import os
import tarfile
import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

#현재 작업공간에 datasets/housing 디렉터리를 만들고 housing.tgz파일을 내려받고 압축을 푸는 함수
def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path) #왼쪽 인수에서 받아와 오른쪽 인수 경로에 저장
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

fetch_housing_data()

In [6]:
import pandas as pd

#데이터를 읽어들이는 함수
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

## 3. 데이터 구조 훑어보기

In [8]:
housing = load_housing_data()
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [9]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [11]:
#ocean_proximity만 object 자료형, 미리보기로 보아선 범주형으로 추정됨
housing["ocean_proximity"].value_counts()

<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: ocean_proximity, dtype: int64